Setup Instructions
==========

These instructions need to be run once to load the requisite libraries for the tutorial.

Jupyter has the concept of _kernels_, which are independent execution environments. They don't
even have to be Python, kernels for other languages exist as well.

By loading a separate kernel for each project, we avoid the complication of different
components/projects having weird interactions, ultimately helping reproducibility.

We first produce a new virtualenv with the libraries we require, then we teach Jupyter
about this new environment with the ipython executable

In [ ]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_11_3_4"
KERNEL_NAME="statistics-das-2023"

set -e
# Get the CMSSW libraries (specifically ROOT)
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ ! -d $CMSSW_VER ]; then
    xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2023/short_exercises/Statistics/${CMSSW_VER}.tar.gz .
    tar -xf ${CMSSW_VER}.tar.gz
fi
pushd ${CMSSW_VER}
scramv1 b ProjectRename
eval `scramv1 runtime -sh`

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
cd $DIR
eval `scramv1 runtime -sh`
cd -
exec python3 "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

Results
=======

If successful, you should see something similar to the following:

```
Loaded CMSSW_10_2_0 into statistics-das!
```

The new kernel you just made will then show up in the various Jupyter dropdowns, allowing you to use it for different notebooks